# PDF reader and embedder via Azure/ Open AI tools for MM-RAG
## 📘 Tanat Piumsuwan's Customized Version

This notebook also contains scripts for reading and extracting content from PDF files.

### ✅ Setup Before Running This Notebook:

- Ensure the **corresponding PDF file** is in its designated folder "\Data"
- Add yourown .env, based on the empty one I provided.

## Environment setup:

In [47]:
%load_ext autoreload
%autoreload 2
import os
from glob import glob

pdf_dir = os.getcwd()

from dotenv import load_dotenv
load_dotenv()

document_basename = os.environ.get("DATA_NAME")
pdf_dir = os.path.join(os.getcwd(),'Data')
document_name = glob(os.path.join(pdf_dir, "*.PDF"))[0]
pdf_doc = os.path.join(pdf_dir,document_name )  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### OpenAI Models information

In [48]:
# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
DI_endpoint = os.environ.get('DI_ENDPOINT')
key = os.environ.get("DI_KEY")

## Azure DI

In [49]:
# import libraries
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, DocumentContentFormat
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest


# helper functions

def get_words(page, line):
    result = []
    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)
    return result


def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (
            word.span.offset + word.span.length
        ) <= (span.offset + span.length):
            return True
    return False



def analyze_layout(pdf_doc):
    # sample document

    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=DI_endpoint, credential=AzureKeyCredential(key)
    )
    # Open your local PDF file
    with open(pdf_doc, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            model_id="prebuilt-layout",
            body=f,
            output_content_format=DocumentContentFormat.MARKDOWN,
            
        )

    result = poller.result()

    print("Extracted keys: ",result.keys())

    return result

In [50]:
document_intelligence_client = DocumentIntelligenceClient(
        endpoint=DI_endpoint, credential=AzureKeyCredential(key)
    )

In [51]:
DI_result = analyze_layout(pdf_doc)

ServiceRequestError: No connection adapters were found for 'aifoundry-service-001/documentintelligence/documentModels/prebuilt-layout:analyze?api-version=2024-11-30&outputContentFormat=markdown'

In [ ]:
DI_result['content']

'<figure>\n</figure>\n\n\nKNOW THE\nMARKETS\n\n\n# STANDPOINT\n\nJ.P.Morgan\nASSET MANAGEMENT\n\n\n<figure>\n\nKAsset\n\n</figure>\n\n\nGlobal Strength. Local Expertise.\n\n<!-- PageFooter="August 2025 | As of 31 July, 2025" -->\n<!-- PageBreak -->\n\n<!-- PageHeader="Global Strength. Local Expertise. สารบัญ" -->\n\nJ.P.Morgan\nASSET MANAGEMENT\n\n\n<figure>\n\nKAsset\n\n</figure>\n\n\n<figure>\n</figure>\n\n\n3\\.\nมุมมองการลงทุนในตราสารหนี้โลกและไทย\n\n4\\.\nสหรัฐฯ : มุมมองเศรษฐกิจและการลงทุน\n\n5\\.\nยุโรป : มุมมองเศรษฐกิจและการลงทุน\n\n6\\.\nญี่ปุ่น : มุมมองเศรษฐกิจและการลงทุน\n\n7\\.\nจีน : มุมมองเศรษฐกิจและการลงทุน\n\n8\\.\nอินเดีย : มุมมองเศรษฐกิจและการลงทุน\n\n9\\.\nเวียดนาม : มุมมองเศรษฐกิจและการลงทุน\n\n10\\.\nไทย : มุมมองเศรษฐกิจและการลงทุน\n\n11\\.\nมุมมองการลงทุนราย Sector\n\n12\\.\nตารางมุมมองการลงทุนเดือนสิงหาคม 2025 สำหรับหุ้น\n\n13\\.\nตารางมุมมองการลงทุนเดือนสิงหาคม 2025 สำหรับหุ้น\n\n14\\. ตารางมุมมองการลงทุนเดือนสิงหาคม 2025 สำหรับตราสารหนี้ สินทรัพย์ทางเลือก และเงิ